In [ ]:
"""
Classification of the Planet SuperDove imagery from the XGBoost model
Author: maxwell.cook@colorado.edu
"""

import os, time
import pandas as pd
import rioxarray as rxr
import xgboost as xgb
import numpy as np
import xarray as xr

# Functions script import
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

print(os.getcwd())

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping'
# homedir = '/home/jovyan/data-store/data/iplant/home/maco4303/data/' # cyverse

print("Imports successful!")

In [ ]:
results_dir = os.path.join(maindir, 'results/xgboost-cv/classification/')
# results_dir = os.path.join(homedir, 'OPP/results/xgboost-cv/classification/') # cyverse
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
print(f"Saving results to {results_dir}")

In [ ]:
# Load the best XGBoost model from cross-validation based on holdout accuracy

In [ ]:
best_fold = 4
best_params = {
    'learning_rate': 0.002185884764883822, 
    'max_depth': 12, 
    'n_estimators': 837, 
    'min_child_weight': 2, 
    'subsample': 0.5014281326674929, 
    'colsample_bytree': 0.9226235922462854, 
    'gamma': 0.8462080004717595
}

In [ ]:
# Load the trained model for best fold
model_fp = os.path.join(results_dir, f'dc-xgb_fold{best_fold}.model')
print(f"Loading model from path: {model_fp}")

xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(np.unique(fold_train_bal['code'])),
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    min_child_weight=best_params['min_child_weight'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    gamma=best_params['gamma'],
    eval_metric=['mlogloss', 'auc', 'merror'], # track multiple evaluation metrics
    early_stopping_rounds=100, # stop if no improvement
    random_state=44
)

xgb_model.load_model(model_fp)
print("Model loaded !")